In [1]:
import numpy as np
import scipy
from scipy.optimize import fsolve

## 1.)

### a.) 
Derive an equation for distance from a station, r, as a function of its observed S − P time.

$V = \frac{r}{t} \rightarrow t = \frac{r}{V}$

$V_{p} = \frac{r}{t_{p}}$ and $V_{s} = \frac{r}{t_{s}}$

$t_{sp} = t_{s} - t_{p}  = \frac{r}{V_{s}} - \frac{r}{V_{p}} = \frac{r\left(V_{p} - V_{s}\right)}{V_{s} V_{p}}$

$r = \frac{V_{s}V_{p}\left(t_{sp} \right)}{V_{p} - V_{s}} $

### b.)  
Solve for the two possible locations for the earthquake. Give your results as the (x, y) km offset from station B

In [2]:
vp = 6.5
vs = 3.6
tspa = 8
tspb = 6

In [3]:
rA = np.round((vp * vs * tspa) / (vp - vs), 2)
rB = np.round((vp * vs * tspb) / (vp - vs), 2)
print(f'Distance from station A: {rA} km, distance from station B: {rB} km')

Distance from station A: 64.55 km, distance from station B: 48.41 km


In [4]:
def circles(x):
    return[
        x[0]**2 + (x[1]-50)**2 - rA**2,
        x[0]**2 + x[1]**2 - rB**2        
    ]

In [5]:
roots = np.round(fsolve(circles, [0,0]),2)
print(f'Event location on the grid: (+/-{roots[0]}, {roots[1]})')

Event location on the grid: (+/-47.93, 6.77)


### c.)  
For each earthquake location, what is the predicted S − P time at Latteville?

In [6]:
# define position of Latteville (Lt), and potential sources
Lt = [100,25]
src1 = [-roots[0], roots[1]]
src2 = [roots[0], roots[1]]
print(f'Locations on the grid - source1: {src1} (offshore), source2: {src2} (onshore)')

Locations on the grid - source1: [-47.93, 6.77] (offshore), source2: [47.93, 6.77] (onshore)


In [7]:
# Define r as linear distance between Latteville and sources on the grid
r1 = np.round(np.sqrt(
    (Lt[0] - src1[0])**2 + (Lt[1] - src1[1])**2
),2)

r2 = np.round(np.sqrt(
    (Lt[0] - src2[0])**2 + (Lt[1] - src2[1])**2
),2)

print(f'Linear distance from Latteville to source1: {r1} km, source2: {r2} km')

Linear distance from Latteville to source1: 149.05 km, source2: 55.17 km


In [8]:
tsp1 = np.round((r1 * (vp - vs)) / (vp*vs), 2)
tsp2 = np.round((r2 * (vp - vs)) / (vp*vs), 2)
print(f'P and S wave spread from source 1 (offshore): {tsp1} s, source 2: {tsp2} s')

P and S wave spread from source 1 (offshore): 18.47 s, source 2: 6.84 s


### d.)  
The strongest ground shaking usually results from S waves. Assuming the offshore location is correct, how much advance warning could be conveyed to Latteville before the S wave arrives at Latteville? Assume that the warning is possible as soon as the S wave has arrived at both stations.

In [9]:
# define s wave arrival time at stations.
s1 = np.round(rA / vs,2)
s2 = np.round(rB / vs,2)
sL = np.round(r1 / vs,2)
print(f'S wave arrivals at A: {s1} s, B: {s2} s, Latteville: {sL} s')
print(f'Available time for warning: {sL - s1} s')

S wave arrivals at A: 17.93 s, B: 13.45 s, Latteville: 41.4 s
Available time for warning: 23.47 s


### e.)  
How much pre-S-wave warning could be provided to Latteville if the offshore earthquake location and origin time were known instantly (e.g., from a dense offshore network of seismometers)?

I suppose it would be the maximum time for the S wave to arrive in Latteville, which is about 41 seconds. Minus the telemetry delay, of course

## 2.)

In [10]:
import numpy as np
M = np.array([
    [-1.8 , -0.38, -0.96],
    [-0.38, -1.9 , 0.62 ],
    [-0.96,  0.62, 3.7  ]
]) * 10**17

### a.)
Compute M0 and MW for this event.Compute M0 and MW for this event.

In [11]:
M_0 = 1/np.sqrt(2) * np.sqrt(sum(M.flatten()**2))
print(f'M0 = {M_0}')

M_W = (2/3) * (np.log10(M_0) - 9.1)
print(f'MW = {M_W}')

M0 = 3.42350697385006e+17
MW = 5.622980811255467


### b.)
Compute the eigenvalues σ1, σ2, and σ3 (sorted such that σ1 > σ2 > σ3) and express M in its principal axes coordinates.

In [12]:
from numpy.linalg import eig

In [13]:
eig_val, eig_vec = np.linalg.eig(M)
print(eig_val)

[ 3.94087273e+17 -2.23977183e+17 -1.70110090e+17]


In [14]:
eig_vals = np.array([eig_val[0], eig_val[2], eig_val[1]])
eig_vals

array([ 3.94087273e+17, -1.70110090e+17, -2.23977183e+17])

In [15]:
m_princ = np.array(
    [
        [eig_vals[0], 0, 0],
        [0, eig_vals[1], 0],
        [0, 0, eig_vals[2]]
    ]
)

In [16]:
m_princ

array([[ 3.94087273e+17,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.70110090e+17,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.23977183e+17]])

### c.)
Compute the parameter (equation 9.17), the measure of the misfit with a double-couple source. Is its value close to that expected for a pure double-couple (DC) source or a pure compensated linear vector dipole (CLVD) source? 

In [17]:
eps = np.round(eig_vals[1] / abs(eig_vals[0]), 2)
print(f'Epsilon = {eps},  pure compensated linear vector dipole (CLVD) source expected')

Epsilon = -0.43,  pure compensated linear vector dipole (CLVD) source expected


### d.)
Decompose M into MDC and MCLV D using Equation (9.14) and compute the scalar moment of each part.

In [18]:
M_dc = np.array(
    [
        [(1/2)*(eig_vals[0] - eig_vals[2]), 0, 0],
        [0, 0, 0],
        [0, 0, (-1/2)*(eig_vals[0] - eig_vals[2])]
    ]
)

M_clvd = np.array(
    [
        [-eig_vals[1] / 2, 0, 0],
        [0, eig_vals[1], 0],
        [0, 0, -eig_vals[1] / 2]
    ]
)

print(M_dc)
print('')
print(M_clvd)

[[ 3.09032228e+17  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.09032228e+17]]

[[ 8.5055045e+16  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.7011009e+17  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  8.5055045e+16]]


In [19]:
M_dc_scalar = 1/np.sqrt(2) * np.sqrt(sum(M_dc.flatten()**2))
M_clvd_scalar = 1/np.sqrt(2) * np.sqrt(sum(M_clvd.flatten()**2))

print(M_dc_scalar)
print(M_clvd_scalar)

3.0903222806802886e+17
1.4731965929674778e+17


### e.)
Devise an alternative decomposition of M into MDC and MCLVD that maximizes the CLVD part and compute the scalar seismic

### f.)
Explain your results in (a), (d), and (e) in terms of your computed  parameter in (c). Hint: copy your lines of codes used to compute the parameters

## 3.)

Simple line of best fit to the data produces an equation to estimate error:

Expected Error = 1.55 * true magnitude + 0.02